In [1]:
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import cleanlab

In [2]:
df = pd.read_csv( os.path.join('csv_label', 'learning_data_10.csv') )
df=df[df['day']<=2]
df

,original,label,time,day
0,53.4,0,15,0
1,52.8,0,15,0
2,52.8,0,15,0
3,53.2,0,15,0
4,53.6,0,15,0
...,...,...,...,...
1151450,53.6,0,6,2
1151451,53.8,0,6,2
1151452,53.1,0,6,2
1151453,53.0,0,6,2


In [3]:
def make_data(x, str):
    x=pd.DataFrame(x)
    window=10
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    
#     for i in range(-1, -window, -1):
#         column=f'{str}_lag{i}'
#         x[column]=x[str].shift(i)   
    return x

lag=[] 
for str in ['original']:
    lag.append(make_data(df[str], str))

# lag.append(df['trend'])
# lag.append(df['seasonal'])
# lag.append(df['resid'])
lag.append(df['day'])
lag.append(df['label'])
df_add_lag=pd.concat(lag, axis=1)
df=df_add_lag.dropna().reset_index(drop=True)
df

,original,original_lag1,original_lag2,original_lag3,original_lag4,original_lag5,original_lag6,original_lag7,original_lag8,original_lag9,day,label
0,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8,53.4,0,0
1,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8,0,0
2,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,0,0
3,55.7,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,0,0
4,55.0,55.7,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1151441,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,56.0,2,0
1151442,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,2,0
1151443,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,2,0
1151444,53.0,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,2,0


In [4]:
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import os
import numpy as np
import support 

def under_sampling(x):
    import warnings
    import pandas as pd
    import os
    warnings.simplefilter('ignore', pd.core.common.SettingWithCopyWarning)
    df = x
    x_majority=df.loc[df.iloc[:,-1]==0]
    x_minority=df.loc[df.iloc[:,-1]==1]
    from sklearn.cluster import KMeans
    km = KMeans(random_state=1, n_clusters=10)
    km.fit(x_majority.iloc[:, :-1])
    x_majority['Cluster'] = km.predict(x_majority.iloc[:, :-1])
    ratio = x_majority['Cluster'].value_counts() / x_majority.shape[0]
    n_sample_ary = (ratio * x_minority.shape[0]).sort_index().astype('int64')
    df_Xs=[]
    for i, n_sample in enumerate(n_sample_ary):
        #重複を許すか許さないかで変わるのか？
        df_Xs.append(x_majority[x_majority['Cluster']==i].sample(n=n_sample, replace=True))

    df_Xs.append(x_minority)
    x=pd.concat(df_Xs, sort=True)
    x=x.drop('Cluster', axis=1)
    
    return x

def calc_avg(x, y):
    import pandas as pd
    import os
    df_x=pd.DataFrame(x)
    df_y=pd.DataFrame(y)
    df_x_raw=df_x[0]
    raw_df=pd.concat([df_x_raw, df_y], axis=1)
    raw_df.columns=range(raw_df.shape[1])
    df_ex=raw_df.loc[raw_df[1]==0]
    N=len(df_ex)
    Leq=10*np.log10(np.sum(np.power(10, df_ex[0]/10)))-10*np.log10(N)
    return Leq

def calc_avg2(x, y):
    import pandas as pd
    import os
    df_x=pd.DataFrame(x)
    df_y=pd.DataFrame(y)
    df_x_raw=df_x[0]
    raw_df=pd.concat([df_x_raw, df_y], axis=1)
    raw_df.columns=range(raw_df.shape[1])
    df_ex=raw_df
    N=len(df_ex)
    Leq=10*np.log10(np.sum(np.power(10, df_ex[0]/10)))-10*np.log10(N)
    return Leq

In [5]:
x = df[ df.columns[ :-1 ] ]
y=  df[ df.columns[ -1 ] ]
va_period_list=list(set(df['day']))

pr=[]
re=[]
f1 = []
ac = []
y_Leq=[]
pred_Leq=[]
no_Leq=[]
n = []

for va_period in va_period_list:
    is_tr=x['day']!=va_period
    is_va=x['day']==va_period
    df_x=x[x.columns[:-1]]
    x_train, x_test = df_x[is_tr].values, df_x[is_va].values
    y_train, y_test = y[is_tr].values, y[is_va].values
    
    print('train:', len(x_train))
    print('test:', len(x_test))

    # 学習データだけダウンサンプリング
    y_data=y_train[:, np.newaxis]
    data=np.concatenate([x_train, y_data], 1)
    train_df=pd.DataFrame(data)
    train_us=under_sampling(train_df)
    index = train_us.index.tolist()
    x_train=x_train[index]
    y_train=y_train[index]

    rcf = RandomForestClassifier(n_estimators=10, max_depth=5, max_features='sqrt', random_state=1234)
    cl = cleanlab.classification.CleanLearning(rcf)
    cl.fit(x_train, y_train)
    z= cl.predict(x_test)
    pr.append( precision_score( y_test, z ) )
    re.append( recall_score( y_test, z) )
    f1.append( f1_score( y_test, z ) )
    ac.append( accuracy_score( y_test, z ) )
    n.append( len( x_test ) / len( x ) )

    #等価騒音レベル計算
    no_Leq.append(calc_avg2(x_test, y_test))
    y_Leq.append(calc_avg(x_test, y_test))
    pred_Leq.append(calc_avg(x_test, z))
    
print( 'CV Score:' )
print( '  Precision Score = %f'%( np.average( pr, weights=n ) ) )
print( '  Recall Score = %f'%( np.average( re, weights=n ) ) )
print( '  F1 Score = %f'%( np.average( f1, weights=n ) ) )
print( '  Accuracy Score = %f'%( np.average( ac, weights=n ) ) )
print( '  no_Leq = %f'%( np.average( no_Leq, weights=n ) ) )
print( '  y_Leq = %f'%( np.average( y_Leq, weights=n ) ) )
print( '  pred_Leq = %f'%( np.average( pred_Leq, weights=n ) ) )

train: 719470
test: 431976
train: 863961
test: 287485
train: 719461
test: 431985
CV Score:
  Precision Score = 0.040098
  Recall Score = 0.680180
  F1 Score = 0.075511
  Accuracy Score = 0.614549
  no_Leq = 57.586446
  y_Leq = 57.370390
  pred_Leq = 48.917495


In [16]:
x = df[ df.columns[ :-1 ] ]
y=  df[ df.columns[ -1 ] ]
va_period_list=list(set(df['day']))

pr=[]
re=[]
f1 = []
ac = []
y_Leq=[]
pred_Leq=[]
no_Leq=[]
n = []

for va_period in va_period_list:
    is_tr=x['day']!=va_period
    is_va=x['day']==va_period
    df_x=x[x.columns[:-1]]
    x_train, x_test = df_x[is_tr].values, df_x[is_va].values
    y_train, y_test = y[is_tr].values, y[is_va].values
    
    print('train:', len(x_train))
    print('test:', len(x_test))

    # 学習データだけダウンサンプリング
    y_data=y_train[:, np.newaxis]
    data=np.concatenate([x_train, y_data], 1)
    train_df=pd.DataFrame(data)
    train_us=under_sampling(train_df)
    index = train_us.index.tolist()
    x_train=x_train[index]
    y_train=y_train[index]

    rcf = RandomForestClassifier(n_estimators=10, max_depth=5, max_features='sqrt', random_state=1234)
    rcf.fit(x_train, y_train)
    z= rcf.predict(x_test)
    pr.append( precision_score( y_test, z ) )
    re.append( recall_score( y_test, z) )
    f1.append( f1_score( y_test, z ) )
    ac.append( accuracy_score( y_test, z ) )
    n.append( len( x_test ) / len( x ) )

    #等価騒音レベル計算
    no_Leq.append(calc_avg2(x_test, y_test))
    y_Leq.append(calc_avg(x_test, y_test))
    pred_Leq.append(calc_avg(x_test, z))
    
print( 'CV Score:' )
print( '  Precision Score = %f'%( np.average( pr, weights=n ) ) )
print( '  Recall Score = %f'%( np.average( re, weights=n ) ) )
print( '  F1 Score = %f'%( np.average( f1, weights=n ) ) )
print( '  Accuracy Score = %f'%( np.average( ac, weights=n ) ) )
print( '  no_Leq = %f'%( np.average( no_Leq, weights=n ) ) )
print( '  y_Leq = %f'%( np.average( y_Leq, weights=n ) ) )
print( '  pred_Leq = %f'%( np.average( pred_Leq, weights=n ) ) )

train: 719470
test: 431976
train: 863961
test: 287485
train: 719461
test: 431985
CV Score:
  Precision Score = 0.041470
  Recall Score = 0.704524
  F1 Score = 0.077928
  Accuracy Score = 0.612463
  no_Leq = 57.586446
  y_Leq = 57.370390
  pred_Leq = 49.090536


In [6]:
x = df[ df.columns[ :-2 ] ].values
y=  df[ df.columns[ -1 ] ].values
print('train:', len(x))
print('test:', len(x))

# # 学習データだけダウンサンプリング
# y_data=y[:, np.newaxis]
# data=np.concatenate([x, y_data], 1)
# train_df=pd.DataFrame(data)
# train_us=under_sampling(train_df)
# index = train_us.index.tolist()
# x_train=x[index]
# y_train=y[index]

rcf = RandomForestClassifier(n_estimators=10, max_features='sqrt', random_state=1234)
rcf.fit(x, y)
z= rcf.predict(x)

    
print( 'Score:' )
print( '  Precision Score = %f'%( precision_score( y, z )) )
print( '  Recall Score = %f'%(  recall_score( y, z) ))
print( '  F1 Score = %f'%( f1_score( y, z )  ) )
print( '  Accuracy Score = %f'%( accuracy_score( y, z ) ) )
print( '  no_Leq = %f'%( calc_avg2(x, y) ) )
print( '  y_Leq = %f'%( calc_avg(x, y) ) )
print( '  pred_Leq = %f'%( calc_avg(x, z) ) )

train: 1151446
test: 1151446
Score:
  Precision Score = 0.998760
  Recall Score = 0.819678
  F1 Score = 0.900401
  Accuracy Score = 0.995513
  no_Leq = 57.788656
  y_Leq = 57.520680
  pred_Leq = 57.543181
